# Zero-shot mutant prediction For VHH

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["http_proxy"] = "http://g5:15777"
os.environ["https_proxy"] = "http://g5:15777"

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

wild_type = f"../example_data/expr/vhh.fasta"
sequence = read_seq(wild_type)

all_mutants = []
for i, wt in enumerate(sequence):
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        if aa != wt:
            mutant = f"{wt}{i+1}{aa}"
            all_mutants.append(mutant)
df = pd.DataFrame({"mutant": all_mutants})

In [2]:
base_model_path = "AI4Protein/vhh_model_"

for i in range(0, 5):
    model_path = f"{base_model_path}{i}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("AI4Protein/Prime_690M", trust_remote_code=True)
    model.eval()
    model = model.to(device)

    tokenized = tokenizer(sequence, return_tensors="pt").to(device)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits[0, 1:-1, :].log_softmax(dim=-1)
    
    scores = []
    for mutant in tqdm(df["mutant"]):
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df[f"model_{i}"] = scores
    print(f"Model {i} done")

  0%|          | 0/2698 [00:00<?, ?it/s]

Model 0 done


  0%|          | 0/2698 [00:00<?, ?it/s]

Model 1 done


  0%|          | 0/2698 [00:00<?, ?it/s]

Model 2 done


  0%|          | 0/2698 [00:00<?, ?it/s]

Model 3 done


  0%|          | 0/2698 [00:00<?, ?it/s]

Model 4 done


In [5]:
aggregate_score = 0
for i in range(0, 5):
    # min-max
    df[f"model_{i}"] = (df[f"model_{i}"] - df[f"model_{i}"].min()) / (df[f"model_{i}"].max() - df[f"model_{i}"].min())
    aggregate_score += df[f"model_{i}"]
df["aggregate"] = aggregate_score
df = df.sort_values("aggregate", ascending=False)

In [6]:
df.head(50)

,mutant,model_0,model_1,model_2,model_3,model_4,aggregate
2130,V113D,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
2216,P117Q,0.998058,0.998056,0.998056,0.998055,0.998052,4.990277
277,A15P,0.996919,0.996917,0.996916,0.996916,0.996912,4.984581
1041,I55T,0.995997,0.995996,0.995996,0.995998,0.995982,4.979968
2113,W112F,0.995173,0.995172,0.995171,0.995170,0.995160,4.975846
547,P29T,0.984755,0.984754,0.984753,0.984752,0.984750,4.923765
1065,A57D,0.982992,0.982990,0.982990,0.982990,0.982981,4.914943
2165,D114Y,0.980867,0.980865,0.980865,0.980863,0.980857,4.904316
1610,N85S,0.977901,0.977900,0.977899,0.977899,0.977893,4.889491
2164,D114W,0.968337,0.968335,0.968335,0.968334,0.968335,4.841677
